<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/PYT001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

大多数机器学习工作流程包括数据处理、创建模型、优化模型参数以及保存训练好的模型。本教程将向您介绍在PyTorch中实现的完整机器学习工作流程，并提供链接以便您进一步学习每个概念。

我们将使用FashionMNIST数据集来训练一个神经网络，这个网络能够预测输入图像属于以下哪一类：T恤/上衣、长裤、套头衫、连衣裙、外套、凉鞋、衬衫、运动鞋、包或短靴。

# 处理数据
PyTorch有两个基本单元来处理数据：torch.utils.data.DataLoader 和 torch.utils.data.Dataset。Dataset 存储样本及其对应的标签，DataLoader 则围绕 Dataset 创建一个可迭代的对象。

在Python中，可迭代对象并不是特指某个具体的数据类型，而是指存储了元素的一个容器对象，这些容器对象内的元素可以通过__iter__()方法或__getitem__()方法进行访问。常见的可迭代对象包括但不限于：列表、元组、字典、集合、字符串等。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch提供了诸如TorchText、TorchVision和TorchAudio等针对特定领域的库，这些库都包含数据集。在本教程中，我们将使用一个TorchVision数据集。

torchvision.datasets模块包含许多真实世界视觉数据集的Dataset对象，如CIFAR、COCO（完整列表参见此处）。在本教程中，我们使用FashionMNIST数据集。每个TorchVision数据集都包含两个参数：transform和target_transform，分别用于修改样本和标签。

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

我们将Dataset作为参数传递给DataLoader。这会在我们的数据集上创建一个可迭代的对象，并支持自动分批、采样、打乱以及多进程数据加载。这里我们定义了一个批大小为64，即DataLoader中的每个元素可迭代地返回一个包含64个特征和标签的批次。

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# 创建模型
要在PyTorch中定义一个神经网络，我们创建一个继承自nn.Module的类。我们在__init__函数中定义网络的层，并在forward函数中指定数据将如何在网络中传递。为了加速神经网络中的操作，我们将其迁移到可用GPU或MPS上。

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


这段代码定义了一个名为`NeuralNetwork`的类，它继承了`nn.Module`，这是所有神经网络模块的基类。在`NeuralNetwork`内部，定义了两个主要的函数：`__init__`和`forward`。

1. `__init__`函数：
   - `__init__`是Python中的一个特殊函数，用于初始化一个类的实例。
   - `super().__init__()`调用了父类（`nn.Module`）的`__init__`方法，完成了一些必要的初始化工作。
   - `self.flatten = nn.Flatten()`创建了一个展平层，它负责将输入数据展平成一维向量。这对于将二维图像数据输入到全连接层进行处理是必需的。
   - `self.linear_relu_stack = nn.Sequential(...)`定义了一个序列模型，它按顺序包含了一些层。这个序列模型包括以下几个部分：
     a. `nn.Linear(28*28, 512)`：一个线性层（全连接层），它将展平后的输入（尺寸为28*28）映射到512个特征。
     b. `nn.ReLU()`：一个激活层，使用ReLU（修正线性单元）激活函数，用于增加模型的非线性。
     c. `nn.Linear(512, 512)`：又一个线性层，将512个输入特征映射到512个输出特征。
     d. `nn.ReLU()`：再次使用ReLU激活函数。
     e. `nn.Linear(512, 10)`：最后一个线性层，将512个输入特征映射到10个输出特征，对应于10个类别的预测得分。

2. `forward`函数：
   - `forward`定义了数据在模型中如何前进。对于所有的nn.Module，每当输入数据传给模型时，forward函数都会被调用。
   - `x = self.flatten(x)`：输入数据`x`被展平。
   - `logits = self.linear_relu_stack(x)`：展平后的输入数据通过先前定义的层序列进行处理，最终得到`logits`，它是模型的输出，代表各个类别的预测得分。
   - 最后，`logits`被返回，代表模型的输出。

3. `model = NeuralNetwork().to(device)`：
   - 这行代码实例化了`NeuralNetwork`类，创建了一个模型实例，并将其发送到指定的`device`（可能是CPU、GPU或MPS，取决于可用性和`device`变量的值）。

4. `print(model)`：
   - 最后，打印模型实例，会输出模型的结构，即各层的排列和参数。

总的来说，这段代码通过定义一个继承自`nn.Module`的类来创建了一个简单的神经网络模型，该模型包含数据展平、线性变换和ReLU激活等操作，并将其实例化且转移到指定计算设备上。

# 优化模型参数
为了训练模型，我们需要一个损失函数和一个优化器。

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在一次单独的训练循环中，模型会对训练数据集（分批输入模型）进行预测，并通过反向传播预测误差来调整模型的参数。

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

这段代码定义了一个名为`train`的函数，用于训练神经网络模型。函数接受四个参数：`dataloader`、`model`、`loss_fn`、和`optimizer`，并执行训练过程。下面是对代码的逐行解释：

1. `def train(dataloader, model, loss_fn, optimizer):`
   - 定义一个名为`train`的函数，它有四个参数：`dataloader`（数据加载器）、`model`（模型）、`loss_fn`（损失函数）、和`optimizer`（优化器）。

2. `size = len(dataloader.dataset)`
   - 获取数据集大小，用于后面打印训练进度。

3. `model.train()`
   - 将模型设置为训练模式。这会启用 dropout 和 batch normalization（批量标准化）等层的训练行为。

4. `for batch, (X, y) in enumerate(dataloader):`
   - 遍历`dataloader`中的数据批。`enumerate`函数提供批次索引`batch`以及批次数据`X`（输入）和`y`（目标/标签）。

5. `X, y = X.to(device), y.to(device)`
   - 将输入数据`X`和标签数据`y`转移到`device`（例如：CPU或GPU）上，以进行更快地数学运算。

6. `# Compute prediction error`
   - 注释表明接下来的代码块将计算预测错误。

7. `pred = model(X)`
   - 将输入数据`X`送入模型`model`进行预测，将输出保存到`pred`。

8. `loss = loss_fn(pred, y)`
   - 使用提供的`loss_fn`（损失函数）计算模型预测`pred`和真实标签`y`之间的损失，并将结果保存到`loss`。

9. `# Backpropagation`
   - 注释表明接下来的代码块将执行反向传播。

10. `loss.backward()`
    - 调用`loss.backward()`进行反向传播，计算模型参数的梯度。

11. `optimizer.step()`
    - 使用优化器根据梯度更新模型参数。

12. `optimizer.zero_grad()`
    - 清空之前的梯度，以准备下一个批次的训练。

13. `if batch % 100 == 0:`
    - 这个条件判断用于确定是否达到了打印训练进度的条件，此处设定每100个批次打印一次信息。

14. `loss, current = loss.item(), (batch + 1) * len(X)`
    - 从`loss`张量中提取实际的标量损失值，并计算当前已处理的数据点总数。

15. `print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")`
    - 打印当前的损失值和训练进度（已处理数据点 / 数据集总大小）。

总结来说，这段代码实现了一个训练循环，通过数据加载器逐批读取数据，执行模型预测、损失计算、梯度反向传播和参数更新等步骤，并定期输出损失信息和训练进度。

我们还会针对测试数据集评估模型的性能，以确保模型正在学习。

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

这段代码定义了一个名为`test`的函数，用于评估训练好的神经网络模型在测试数据集上的性能。函数接受三个参数：`dataloader`（测试数据的加载器），`model`（要评估的模型），以及`loss_fn`（损失函数，用于计算模型预测和真实标签之间的误差）。接下来对代码进行逐行解释：

1. `size = len(dataloader.dataset)`
   - 获取测试数据集的大小，即数据点总数。

2. `num_batches = len(dataloader)`
   - 获取测试数据集的批次数。

3. `model.eval()`
   - 将模型设置为评估模式。这会禁用 dropout 和 batch normalization（批量标准化）等层的训练特定行为，这些行为在测试时是不需要的。

4. `test_loss, correct = 0, 0`
   - 初始化两个变量：`test_loss` 用于累加测试损失，`correct` 用于累加正确预测的样本数。

5. `with torch.no_grad():`
   - 使用`torch.no_grad()`上下文管理器临时设置所有`requires_grad=True`的张量的梯度计算为不必要，从而减少内存消耗和加快计算速度。在测试阶段通常这么做，因为不需要进行梯度计算。

6. `for X, y in dataloader:`
   - 遍历测试数据加载器中的数据批。

7. `X, y = X.to(device), y.to(device)`
   - 将输入数据`X`和标签数据`y`转移到`device`（例如：CPU或GPU）上，以进行更快地数学运算。

8. `pred = model(X)`
   - 将输入数据`X`送入模型`model`进行预测，将输出保存到`pred`。

9. `test_loss += loss_fn(pred, y).item()`
   - 计算模型预测`pred`和真实标签`y`之间的损失，并将其加到`test_loss`变量上。

10. `correct += (pred.argmax(1) == y).type(torch.float).sum().item()`
    - 通过 `pred.argmax(1)` 获取预测类别，并与真实标签`y`进行比较以计算正确预测的数量。转换结果为`torch.float`类型后求和，然后将结果累加到`correct`变量上。

11. `test_loss /= num_batches`
    - 将累加的测试损失平均值计算出来。

12. `correct /= size`
    - 将正确预测的比例（准确率）计算出来。

13. `print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")`
    - 打印测试结果的汇总，包括准确率和平均损失。

总结来说，这段代码实现了一个测试循环，用于评估模型在测试数据集上的性能，通过计算平均损失和准确率来衡量模型的表现，并输出结果。

训练过程会进行多次迭代（周期）。在每个周期中，模型学习参数以进行更好的预测。我们在每个周期打印模型的准确率和损失；我们希望随着每个周期的进行，准确率不断增加，而损失不断减少。

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308407  [   64/60000]
loss: 2.296837  [ 6464/60000]
loss: 2.277592  [12864/60000]
loss: 2.268646  [19264/60000]
loss: 2.255244  [25664/60000]
loss: 2.224638  [32064/60000]
loss: 2.240944  [38464/60000]
loss: 2.207997  [44864/60000]
loss: 2.197717  [51264/60000]
loss: 2.171117  [57664/60000]
Test Error: 
 Accuracy: 28.5%, Avg loss: 2.165487 

Epoch 2
-------------------------------
loss: 2.175849  [   64/60000]
loss: 2.166821  [ 6464/60000]
loss: 2.115936  [12864/60000]
loss: 2.126971  [19264/60000]
loss: 2.081703  [25664/60000]
loss: 2.019896  [32064/60000]
loss: 2.052705  [38464/60000]
loss: 1.977847  [44864/60000]
loss: 1.974804  [51264/60000]
loss: 1.912145  [57664/60000]
Test Error: 
 Accuracy: 55.6%, Avg loss: 1.909022 

Epoch 3
-------------------------------
loss: 1.933957  [   64/60000]
loss: 1.907794  [ 6464/60000]
loss: 1.802922  [12864/60000]
loss: 1.838940  [19264/60000]
loss: 1.733566  [25664/60000]
loss: 1.683337  [32064/600

这段代码设置了一个训练循环，用于训练神经网络模型并通过测试数据评估其性能。下面是对代码的逐行解释：

1. `epochs = 5`
   - 定义一个变量`epochs`并将其设置为5，表示模型将训练5个周期。一个周期是指模型对所有训练数据进行一次完整的遍历。

2. `for t in range(epochs):`
   - 使用`range(epochs)`创建一个迭代器，它将产生从0到4的整数（总共5个周期）。变量`t`代表当前的周期索引。

3. `print(f"Epoch {t+1}\n-------------------------------")`
   - 打印当前周期的编号（`t+1`，因为`t`是从0开始的），后面跟随一个分隔线用于日志输出格式的清晰易读。

4. `train(train_dataloader, model, loss_fn, optimizer)`
   - 调用先前定义的`train`函数来执行一个周期的模型训练。传入的参数包括训练数据加载器`train_dataloader`、模型`model`、损失函数`loss_fn`和优化器`optimizer`。

5. `test(test_dataloader, model, loss_fn)`
   - 在每个周期训练完成后，调用`test`函数在测试数据集上评估模型性能。传入的参数包括测试数据加载器`test_dataloader`、模型`model`和损失函数`loss_fn`。

6. `print("Done!")`
   - 当所有周期都完成训练和测试后，打印“Done!”表示整个过程已完成。

总结来说，这段代码通过多次迭代（周期）来训练模型，并在每个周期结束后评估模型在测试数据集上的性能，最后输出“Done!”表示训练和测试过程的结束。

# 保存模型
一种常见的保存模型的方式是序列化内部状态字典（包含模型参数）。

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# 加载模型
加载模型的过程包括重建模型结构并将其状态字典加载到其中。

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

这段代码展示了如何在PyTorch中加载一个先前训练好的模型。具体操作分为两步：

1. `model = NeuralNetwork().to(device)`
   - 这一行代码首先实例化了一个名为`NeuralNetwork`的类。这个类应当在之前的代码中定义过，代表了一个具体的神经网络结构。之后，`.to(device)`方法被调用，其作用是将这个模型转移到由`device`变量所指定的硬件上（例如，CPU、GPU）。这是为了确保模型在指定硬件上运行，以获得更优的计算性能。

2. `model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))`
   - 这一行代码做了以下几件事情：
     - `torch.load("model.pth", map_location=device)` 从名为 `"model.pth"` 的文件中加载了模型的状态字典（state dictionary）。状态字典是一个包含了模型所有可学习参数（如权重和偏置）的字典。参数`map_location`用来指定加载模型参数时使用的设备，这保证了模型参数能够被加载到正确的设备上。
     - `model.load_state_dict(...)` 将加载的状态字典加载（拷贝）到第一步创建的`model`实例的对应参数中。这样，模型结构中所有层的参数就被替换为之前保存的状态，也就是说模型现在拥有了与保存时相同的行为。
     - 参数`weights_only=True`指定仅加载模型的参数（权重和偏置等），不加载训练状态（如epoch数或优化器状态）。

总结来说，这段代码通过重建模型结构并将先前保存的模型参数加载到该结构中，实现了模型的加载。这样，我们就可以使用该模型进行进一步的训练或推理任务了。

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


这段代码示例在测试或验证一个已经训练好的模型时，如何对其中的一个样本进行预测，并与实际标签进行比较。以下是对代码的逐行解释：

1. `classes = [ ... ]`
   - 定义了一个名为`classes`的列表，包含了模型可能预测的所有类别标签的字符串形式。这个列表与模型输出层的类别索引相对应，使得模型的输出可以被转换为可读的标签。

2. `model.eval()`
   - 将模型设置为评估模式（`eval`模式）。这个调用会切换比如 dropout、批标准化（batch normalization）等层的行为，使得它们在评估模式下以测试时的预期方式运行。

3. `x, y = test_data[0][0], test_data[0][1]`
   - 从测试数据集`test_data`中提取第一个样本。`x`是模型的输入数据，`y`是相应的实际类别标签（目标值）。这里假设`test_data`是一个包含了（输入，目标）对的元组的列表或数据加载器。

4. `with torch.no_grad():`
   - `torch.no_grad()`上下文管理器被用来暂时设置梯度计算为非必需。在评估模型时，我们不需要计算梯度，因此这可以减少内存消耗并提升速度。

5. `x = x.to(device)`
   - 将输入数据`x`转移到`device`变量指定的设备上（例如，CPU或GPU），以确保模型运行在正确的设备上。

6. `pred = model(x)`
   - 将输入数据`x`送入模型`model`进行预测。模型返回的`pred`是模型的输出，通常是一个由预测得分组成的张量，每个得分对应于`classes`列表中的一个类别。

7. `predicted, actual = classes[pred[0].argmax(0)], classes[y]`
   - `pred[0].argmax(0)`从模型输出中获取预测最大概率所对应的类别索引。因为`pred`可能是包含多个样本输出的批次，所以使用`[0]`来获取第一个样本的预测结果。`argmax(0)`函数返回该样本在所有类别中预测得分最高的索引。
   - 使用这个索引以及实际的标签`y`，分别从`classes`列表中获取预测类别和实际类别的可读字符串形式，分别存储在`predicted`和`actual`变量中。

8. `print(f'Predicted: "{predicted}", Actual: "{actual}"')`
   - 打印出预测的类别标签和实际的类别标签，以比较模型的预测和真实情况。

总结来说，这段代码展示了如何在评估模式下对单个测试样本进行预测，并通过与实际类别标签比较，来直观地查看模型的预测效果。